In [3]:
!pip install bpemb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.9 MB/s 


In [1]:
from bpemb import BPEmb
import difflib
import numpy as np
import re

In [2]:
en = BPEmb(lang="en", vs = 10000)
es = BPEmb(lang="es", vs = 10000)
fr = BPEmb(lang="fr", vs = 10000)
de = BPEmb(lang='de', vs = 10000)

en_words = set([item.replace('▁','') for item in en.words])
es_words = set([item.replace('▁','') for item in es.words])
fr_words = set([item.replace('▁','') for item in fr.words])
de_words = set([item.replace('▁','') for item in de.words])

all_words = [en_words,es_words,fr_words, de_words]

In [3]:
with open('moby.txt','r') as book:
    full_text = book.readlines()

In [4]:
text = ''.join(full_text)

In [5]:
def rand_lang():
  return all_words[np.random.randint(len(all_words))]


#text = '''
#these sentences, formed without bigger words.
#'''
text = ''.join([c if c.isalnum() else f' {c} ' for c in text])

new_text = []

In [8]:
from tqdm import tqdm

In [9]:
word = ''
for word in tqdm(text.split(' ')):
    new_word = ''
    try:
        if len(word)<=4:
            if word.isalnum():
                if np.random.rand()>0.8:
                    new_word = difflib.get_close_matches(word,rand_lang(),n=5)[-1]

            else:
                new_word = word

        elif len(word)>4:
            chunks = [word[:len(word)//2], word[len(word)//2:]]
            joyce_chunks = chunks[:]

            if np.random.rand()>0.5:
                #Change first half of word by word that ends with that half
                candidates = [item for item in rand_lang() if item.endswith(joyce_chunks[0]) and item!=joyce_chunks[0]]
                if candidates:
                  joyce_chunks[0] = candidates[0]

            elif np.random.rand()>0.4:
                #Change second half of word by similar
                rand_language = all_words[np.random.randint(len(all_words))]
                joyce_chunks[1] = difflib.get_close_matches(joyce_chunks[1],rand_lang(),n=5)[-1]

            new_word = ''.join(joyce_chunks)
            if np.random.rand()>0.8:
                new_word = difflib.get_close_matches(word,rand_lang(),n=5)[-1]
            #new_text.append(new_word)
        
        elif len(word)>9 and np.random.rand()>0.85:
            chunks = [word[:len(word)//3], word[len(word)//3:2*len(word)//3], word[2*len(word)//3:]]
            joyce_chunks = chunks[:]

            if np.random.rand()>0.5:
                #Change first third of word by word that ends with that half
                candidates = [item for item in rand_lang() if item.endswith(joyce_chunks[0]) and item!=joyce_chunks[0]]
                if candidates:
                  joyce_chunks[0] = candidates[0]

            elif np.random.rand()>0.4:
                #Change second or third part of word by similar
                rand_language = all_words[np.random.randint(len(all_words))]
                part = np.random.choice([1,2])
                joyce_chunks[part] = difflib.get_close_matches(joyce_chunks[part],rand_lang(),n=5)[-1]

            new_word = ''.join(joyce_chunks)

            if np.random.rand()>0.9:
                new_word = difflib.get_close_matches(word,rand_lang(),n=5)[-1]

        if new_word!='':
          new_text.append(new_word)
    except:
      new_text.append(word)


100%|██████████| 701263/701263 [34:33<00:00, 338.17it/s]


In [11]:
final_text = ''
for c in new_text:
  if c.isalnum():
    final_text+=f' {c}'
  else:
    final_text+=f'{c} '

In [13]:
with open('moby_joyced.txt','w') as newfile:
    newfile.write(final_text)